# Script To Get Commit Differences Between Two Commits for the GC Authors 

## Set Variables

In [44]:
string pathToRepository =  ""; // Fill in e.g., @"C:\Users\musharm\source\repos\runtime";
string baselineCommit = "2d7eea2";
string comparandCommit = "9e6ba1f";
string outputPath = ""; // Fill in e.g., "C:\\Users\\musharm\\Desktop\\output.md";

## Instantiate Methods

In [54]:
using System.Diagnostics;
using System.IO;

const string gitCommand = @"log --author=""^Maoni Stephens"" --author=""^Andrew Au"" --author=""^Mark Plesko"" --author=""^Mukund Raghav Sharma"" --author=""^Manish Godse"" --pretty=format:""Date: %ad%nAuthor: %an%nTitle: %s%n""";

public void SaveMarkdownTable(string repositoryPath, string baselineCommit, string comparandCommit, string outputPath)
{
    (string output, string error) = RunGitCommand(repositoryPath, baselineCommit, comparandCommit);
    if (!string.IsNullOrEmpty(error))
    {
        Console.WriteLine("Failed with error: " + error);
        return;
    }

    string table = CreateMarkdownTable(output);
    SaveMarkdownTable(table, outputPath);
}

public (string? output, string? error) RunGitCommand(string repositoryPath, string baselineCommit, string comparandCommit)
{
    ProcessStartInfo startInfo = new ProcessStartInfo();
    startInfo.FileName = "git";
    startInfo.Arguments = gitCommand + " " + baselineCommit + "..." + comparandCommit;
    startInfo.WorkingDirectory = repositoryPath;
    startInfo.RedirectStandardOutput = true;
    startInfo.RedirectStandardError = true;
    startInfo.UseShellExecute = false;

    Process process = new Process();
    process.StartInfo = startInfo;
    process.Start();

    string output = process.StandardOutput.ReadToEnd();
    string error = process.StandardError.ReadToEnd();
    process.WaitForExit();

    return (output, error);
}

public string CreateMarkdownTable(string details)
{
    string[] lines = details.Split("\n\n");

    StringBuilder table = new StringBuilder();
    table.AppendLine("| Date | Author | Title | PR |");
    table.AppendLine("|------|--------|-------|----|");

    foreach (string line in lines)
    {
        string[] fields = line.Split("\n");

        string date = fields[0].Replace("Date: ", "");
        string author = fields[1].Replace("Author: ", "");
        string title = fields[2].Replace("Title: ", "");

        // Extract number in parentheses from the title
        string pr = "";
        int startIndex = title.IndexOf('(');
        int endIndex = title.IndexOf(')');
        if (startIndex != -1 && endIndex != -1)
        {
            pr = title.Substring(startIndex + 1, endIndex - startIndex - 1);
        }

        // Create markdown link for PR
        string prLink = $"[{pr}](https://github.com/dotnet/runtime/pull/{pr.Replace("#", "")})";

        table.AppendLine($"| {date} | {author} | {title} | {prLink} |");
    }

    return table.ToString();
}

// 
public void SaveMarkdownTable(string content, string path)
{
    File.WriteAllText(path, content);
}


## Run

In [55]:
SaveMarkdownTable(pathToRepository, baselineCommit, comparandCommit, outputPath);

In [2]:
System.Diagnostics.Process.GetCurrentProcess().Id
#!about

16056

.NET Interactive© 2020 Microsoft CorporationVersion: 1.0.522904+cdfa48b2ea1a27dfe0f545c42a34fd3ec7119074Library version: 1.0.0-beta.24229.4+cdfa48b2ea1a27dfe0f545c42a34fd3ec7119074Build date: 2024-05-01T22:47:43.3565220Zhttps://github.com/dotnet/interactive
